In [1]:
import os 
import codecs
from lxml import etree

# Split cn_dict.txt into small pieces

In [2]:
# This function split a big file into small pieces through lines 

splitLen = 100         # 100 lines per file
outputBase = 'cn_dict_' # output.1.txt, output.2.txt, etc.

with open('../CN_Dict/cn_dict.txt', 'r') as input:

    count = 0
    at = 0
    dest = None
    for line in input:
        if count % splitLen == 0:
            if dest: dest.close()
            dest = open('../CN_Dict/' + outputBase + str(at) + '.txt', 'w')
            at += 1
        dest.write(line)
        count += 1
    
    dest.close()

# Format urls 

In [2]:
# every word in cn_dict.txt will generate a url 
# first curl this web page base on this url to get the number of pages of this web page
# for each page format a url get a urls.txt

# for i in range(0,35) : 
for i in range(33, 35) :
    
    input_fname = '../CN_Dict/cn_dict_' + str(i) + '.txt'
    output_fname = './urls/urls_' + str(i) + '.txt'
    
    print input_fname
    
    with open(input_fname) as cn_dict :
        for line in cn_dict:
        # line = cn_dict.readline()
            init_url = "http://cjjc.weblio.jp/sentence/content/"
            concat_url = init_url + line.strip()
            print concat_url
            !curl -o init.html $concat_url

            f=codecs.open("./init.html","r","utf-8")
            content=f.read()
            f.close()
            tree=etree.HTML(content)

            try:
                numOfPages = int( tree.xpath("//div[@class='kiji']/a[last()-1]")[0].text )
                with open(output_fname, 'a') as fout :
                    fout.write(concat_url + '\n')
                    for i in range(2,numOfPages + 1) :
                        url = concat_url + '/' + str(i) 
                        # print url
                        fout.write(url + '\n')
            except IndexError:
                with open(output_fname, 'a') as fout :
                    fout.write(concat_url + '\n')
      

../CN_Dict/cn_dict_33.txt
http://cjjc.weblio.jp/sentence/content/嘀
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32237    0 32237    0     0  17036      0 --:--:--  0:00:01 --:--:-- 17029
http://cjjc.weblio.jp/sentence/content/幔
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24758    0 24758    0     0  14824      0 --:--:--  0:00:01 --:--:-- 14825
http://cjjc.weblio.jp/sentence/content/镀
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33191    0 33191    0     0  55047      0 --:--:-- --:--:-- --:--:-- 55043
http://cjjc.weblio.jp/sentence/content/舔
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                   

In [24]:
# filenames = ['./urls/urls_20.txt', './urls/urls_21.txt', './urls/urls_22.txt', './urls/urls_23.txt', './urls/urls_24.txt', 
#              './urls/urls_25.txt', './urls/urls_26.txt', './urls/urls_27.txt', './urls/urls_28.txt', './urls/urls_29.txt',
#              './urls/urls_30.txt', './urls/urls_31.txt', './urls/urls_32.txt', './urls/urls_33.txt', './urls/urls_34.txt']

with open('./urls/urls_20_34.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

# Extract chinese & janpanese From the source file

In [25]:
cat ./cn_source/cn_20_34.txt | sed 's/<[^<>]*>//g' | awk -F"<" '{print $1}' > ./cn_source/cn_20_34_pro.txt
cat ./jp_source/jp_20_34.txt | sed 's/<[^<>]*>//g' | awk -F"-" '{print $1}' > ./jp_source/jp_20_34_pro.txt

In [28]:
from itertools import izip
with open('./cn_source/cn_20_34_pro.txt', 'r') as cn, \
    open('./jp_source/jp_20_34_pro.txt', 'r') as jp, \
    open('./cn_jp_pro/cn_jp_20_34.txt', 'w') as cn_jp :
        for line1, line2 in izip(cn, jp):
            cn_jp.write(line1.strip() + '\t' + line2)

In [29]:
# Concat all the files 
filenames = ['./cn_jp_pro/cn_jp_0.txt', './cn_jp_pro/cn_jp_1.txt', './cn_jp_pro/cn_jp_2.txt', './cn_jp_pro/cn_jp_3.txt', 
             './cn_jp_pro/cn_jp_4.txt', './cn_jp_pro/cn_jp_5_9.txt', './cn_jp_pro/cn_jp_10_19.txt', './cn_jp_pro/cn_jp_20_34.txt']
with open('./cn_jp_pro/CN_JP_PRO.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [1]:
import codecs 

from tempfile import mkstemp
from shutil import move
from os import remove


def replace(source_file_path, pattern, substring):
    fh, target_file_path = mkstemp()

    with codecs.open(target_file_path, 'w', 'utf-8') as target_file:
        with codecs.open(source_file_path, 'r', 'utf-8') as source_file:
            for line in source_file:
                target_file.write(line.replace(pattern, substring))
    remove(source_file_path)
    move(target_file_path, source_file_path)

In [6]:
pattern_1 = '&lt;'
substring_1 = '<'

pattern_2 = '&gt'
substring_2 = '>'

pattern_3 = '&amp'
substring_3 = '&'

pattern_4 = '&;'
substring_4 = '&'

with codecs.open('./cn_jp_pro/CN_JP_Uniq.txt', 'r') as source_file:
    with codecs.open('./cn_jp_pro/CN_JP_Pro.txt', 'w') as target_file:
        for line in source_file:
            line = line.replace(pattern_1, substring_1)
            line = line.replace(pattern_2, substring_2)
            line = line.replace(pattern_3, substring_3)
            line = line.replace(pattern_4, substring_4)
            target_file.write(line)